In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Verileri okumak
train_data = pd.read_csv("/kaggle/input/veri-bilimi-ve-yapay-zeka-okulu-kaggle-yarismasi/train.csv")
test_data = pd.read_csv("/kaggle/input/veri-bilimi-ve-yapay-zeka-okulu-kaggle-yarismasi/test.csv")
submission_data = pd.read_csv("/kaggle/input/veri-bilimi-ve-yapay-zeka-okulu-kaggle-yarismasi/sample_submission.csv")

# İlk 5 satırı görüntüleyelim
train_data.head(), submission_data.head()


In [ ]:
test_data.head()

In [ ]:
# Eksik verileri kontrol edelim
train_missing = train_data.isnull().sum()
test_missing = test_data.isnull().sum()

# Temel istatistikler
train_stats = train_data.describe()
test_stats = test_data.describe()

train_missing, test_missing, train_stats, test_stats


In [ ]:
# Eksik verileri doldurmak için strateji: 
# Sürekli değişkenler için ortalama ile doldurma, kategorik değişkenler için mod ile doldurma

# Sayısal sütunlar için eksik değerleri ortalama ile doldur
numeric_columns = train_data.select_dtypes(include=['int64', 'float64']).columns
train_data[numeric_columns] = train_data[numeric_columns].fillna(train_data[numeric_columns].mean())
test_data[numeric_columns] = test_data[numeric_columns].fillna(test_data[numeric_columns].mean())

# Kategorik sütunlar için eksik değerleri mod ile doldur
categorical_columns = train_data.select_dtypes(include=['object']).columns
train_data[categorical_columns] = train_data[categorical_columns].fillna(train_data[categorical_columns].mode().iloc[0])
test_data[categorical_columns] = test_data[categorical_columns].fillna(test_data[categorical_columns].mode().iloc[0])



In [ ]:
# Eksik veriler temizlendikten sonra tekrar kontrol edelim
train_missing_after = train_data.isnull().sum().sum()
test_missing_after = test_data.isnull().sum().sum()

train_missing_after, test_missing_after


In [ ]:
# Eğitim setine one-hot encoding uygula
train_data_encoded = pd.get_dummies(train_data, columns=categorical_columns)

# Test setine one-hot encoding uygula
test_data_encoded = pd.get_dummies(test_data, columns=categorical_columns)

# Test setini eğitim seti ile aynı sütunlara sahip olacak şekilde yeniden düzenle
train_data_encoded, test_data_encoded = train_data_encoded.align(test_data_encoded, join='left', axis=1)

# Eksik sütunları doldur
test_data_encoded = test_data_encoded.fillna(0)
# Dönüştürülmüş veri setini kontrol et
print(train_data_encoded.head())
print(test_data_encoded.head())

In [ ]:
from sklearn.ensemble import RandomForestClassifier


# Hedef değişkeni ve özellikleri ayırın
X_train = train_data_encoded.drop(columns=['value_increased'])  # Özellikler
y_train = train_data_encoded.loc[: ,'value_increased']  # Hedef değişken

X_test = test_data_encoded.drop(columns=['value_increased'])  # Özellikler
y_test = test_data_encoded.loc[: ,'value_increased']


# Modeli oluşturun
model_random_forest = RandomForestClassifier( random_state=42)

# Modeli eğitin
model_random_forest.fit(X_train, y_train)

# Test setinde tahmin yapın
predictions = model_random_forest.predict(X_test)


# Modeli değerlendirin
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Skoru: {f1}")


In [ ]:
# KNN and SVM Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# KNN Model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Skoru: {f1}")
print(f"KNN Doğruluk: {knn_accuracy}")


In [ ]:
# Decision Tree and Random Forest Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# Karar Ağacı Modeli
tree_model = DecisionTreeClassifier(random_state=42,criterion='entropy',max_depth=7)
tree_model.fit(X_train, y_train)
tree_pred = tree_model.predict(X_test)
tree_accuracy = accuracy_score(y_test, tree_pred)
print(f"Karar Ağacı Doğruluk: {tree_accuracy}")
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Skoru: {f1}")
conf_matrix = confusion_matrix(y_test, tree_pred)
print(f"Confusion Matrix:{conf_matrix}")

In [ ]:
# Random Forest Modeli
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Skoru: {f1}")
print(f"Random Forest Doğruluk: {rf_accuracy}")

In [ ]:
# Naive Bayes Model
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# Naive Bayes Modeli
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Skoru: {f1}")
print(f"Naive Bayes Doğruluk: {nb_accuracy}")


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# Hedef değişkeni ve özellikleri ayırın
X_train = train_data_encoded.drop(columns=['value_increased'])  # Özellikler
y_train = train_data_encoded.loc[: ,'value_increased']  # Hedef değişken

X_test = test_data_encoded.drop(columns=['value_increased'])  # Özellikler
y_test = test_data_encoded.loc[: ,'value_increased']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Lojistik Regresyon Modeli Eğitimi
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train_scaled, y_train)

# 5. Model ile tahmin yapma
y_pred = logreg_model.predict(X_test_scaled)

# 6. Sonuçları değerlendirme (accuracy ve classification_report)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Skoru: {f1}")

In [ ]:
# Uzunlukları eşitledikten sonra tahminleri ekleyin
submission_data['value_increased'] = predictions

# Sonuçları CSV dosyasına yazın
submission_data.to_csv('final2_submission.csv', index=False)

In [ ]:
# Uzunlukları eşitledikten sonra tahminleri ekleyin
submission_data['value_increased'] = y_pred

# Sonuçları CSV dosyasına yazın
submission_data.to_csv('final2_submission.csv', index=False)


In [ ]:
# Uzunlukları eşitledikten sonra tahminleri ekleyin
submission_data['value_increased'] = knn_pred

# Sonuçları CSV dosyasına yazın
submission_data.to_csv('final3_submission.csv', index=False)

In [ ]:
# Uzunlukları eşitledikten sonra tahminleri ekleyin
submission_data['value_increased'] = nb_pred

# Sonuçları CSV dosyasına yazın
submission_data.to_csv('final4_submission.csv', index=False)